# **Task 1 | Mask recgognition: Classify People Wearning masks**

___
___

## **1. INITIALIZATION**

### *1.1. IMPORTS*

In [1]:
from IPython.display import display
from xml.dom import minidom

import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import time

from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import torch
import torch.nn as nn

In [2]:
# To fill the requirement.txt file we use the following line of code:
import session_info
session_info.show()

### *1.2. DATA LOADING*

In [3]:
data_root_dir = "data/FaceMaskDetection/"
annotations_dir = data_root_dir + "annotations/"
images_dir = data_root_dir + "images/"

In [4]:
def get_annotations(annotations_dir):
    
    files = os.listdir(annotations_dir)
    files.sort()

    data = []
    mask_value = {"without_mask": -1, "mask_weared_incorrect": 0, "with_mask": 1}

    for file in files:
    
        ann = minidom.parse(annotations_dir)
        img_id = int(ann.getElementsByTagName("filename")[0].firstChild.data[12:-4])
        img_height = int(ann.getElementsByTagName("height")[0].firstChild.data)
        img_width = int(ann.getElementsByTagName("width")[0].firstChild.data)
    
        for box_id,object in enumerate(ann.getElementsByTagName("object")):

            label = mask_value[object.getElementsByTagName("name")[0].firstChild.data]
            xmin = object.getElementsByTagName("xmin")[0].firstChild.data
            xmax = object.getElementsByTagName("xmax")[0].firstChild.data
            ymin = object.getElementsByTagName("ymin")[0].firstChild.data
            ymax = object.getElementsByTagName("ymax")[0].firstChild.data

            data.append((img_id, img_height, img_width, box_id, label, xmin, xmax, ymin, ymax))
    
    columns = ["img_id", "img_height", "img_width", "box_id", "label", "xmin", "xmax", "ymin", "ymax"]
        
    return pd.DataFrame(data=data, columns=columns, index=None)

In [5]:
annotations = get_annotations(data_root_dir)
display(annotations)

PermissionError: [Errno 13] Permission denied: 'data/FaceMaskDetection/'

In [ ]:
class FaceMaskDataset(Dataset):

    def __init__(self, annotations, imgs_dir):
        self.annotations = annotations
        self.img_dir = imgs_dir
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        img = plt.imread(self.img_dir+"maksssksksss{}.png".format(int(self.annotations.iloc[idx]["img_id"])))[:,:,:3]
        xmin = self.annotations.iloc[idx]["xmin"]
        xmax = self.annotations.iloc[idx]["xmax"]
        ymin = self.annotations.iloc[idx]["ymin"]
        ymax = self.annotations.iloc[idx]["ymax"]
        img = transforms.Resize((64,64))(torch.Tensor(img[ymin:ymax,xmin:xmax]).permute(2,0,1))
        label = torch.Tensor([int(self.annotations.iloc[idx]["label"])])
        return img, label

___

## **2. THE MODEL**

We use the [Cascade Classifier](https://docs.opencv.org/3.4/db/d28/tutorial_cascade_classifier.html) provided by OpenCV, which has already been trained. To use it download the trained parameters in the models folder, from the [official source](https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml).

In [12]:
model = cv2.CascadeClassifier("./models/haarcascade_frontalface_default.xml")

In [ ]:
img = cv2.imread(imgs_dir + "maksssksksss0.png")

___

## **3. TESTING THE MODEL**

### *3.1. ACCURACY*

### *3.2. VISUALIZATION*